In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

C:\Users\adhva\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
url = 'https://docs.google.com/spreadsheets/d/1_y0_LJmSY6sNx8qd51T70n0oa_ugN50AVFKuJmXO1-s/export?format=csv&gid=2042605142'
approval = pd.read_csv(url)
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178.0,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184.0,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,90357,294,McLaughlin,1989.0,America's New Majority Project,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,https://americasnewmajorityproject.com/wp-cont...,https://americasnewmajorityproject.com/wp-cont...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,48.0,43.9
99,90363,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,43.0,41.0
100,90390,921,Clarity,NaN,NaN,NaN,NaN,NaN,Clarity Campaign Labs,Online Opt-In Panel,...,https://static1.squarespace.com/static/61ae8d5...,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,51.0,43.0
101,90391,1292,Hart/POS,576.0,CNBC,NaN,NaN,NaN,Hart Research Associates/Public Opinion Strate...,NaN,...,https://pos.org/wp-content/uploads/2025/08/250...,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,49.0,44.0


In [6]:
approval = approval[~approval['poll_id'].isna()]
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178.0,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184.0,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,90357,294,McLaughlin,1989.0,America's New Majority Project,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,https://americasnewmajorityproject.com/wp-cont...,https://americasnewmajorityproject.com/wp-cont...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,48.0,43.9
99,90363,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,43.0,41.0
100,90390,921,Clarity,NaN,NaN,NaN,NaN,NaN,Clarity Campaign Labs,Online Opt-In Panel,...,https://static1.squarespace.com/static/61ae8d5...,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,51.0,43.0
101,90391,1292,Hart/POS,576.0,CNBC,NaN,NaN,NaN,Hart Research Associates/Public Opinion Strate...,NaN,...,https://pos.org/wp-content/uploads/2025/08/250...,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,49.0,44.0


In [7]:
approval.columns.values

array(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'display_name', 'mode', 'state',
       'start_date', 'end_date', 'internal', 'partisan', 'question_id',
       'sample_size', 'population', 'subpopulation', 'population_full',
       'tracking', 'created_at', 'updated_at', 'notes', 'url',
       'url_article', 'url_topline', 'url_crosstab', 'race_id', 'cycle',
       'office_type', 'seat_name', 'election_date', 'stage', 'dem', 'rep'],
      dtype=object)

In [8]:
# morris_to_silver_dict = {
#     'HarrisX/Harris Poll':'Harris Insights & Analytics',
#     'Clarity': 'Clarity Campaign Labs',
#     'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
#     'Monmouth': 'Monmouth University',
#     'HarrisX': 'Harris Insights & Analytics',
#     'UMass Amherst/YouGov': 'YouGov',
#     'Emerson': 'Emerson College',
#     'Marquette Law School': 'Marquette University Law School',
#     'NewsNation/Decision Desk HQ': 'NewsNation',
#     'CNN/SSRS': 'SSRS',
#     'Suffolk': 'Suffolk University',
#     'Harris Poll': 'Harris Insights & Analytics',
#     'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
#     'Winston': 'The Winston Group',
#     'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
#     'Quinnipiac': 'Quinnipiac University',
#     'Siena/NYT': 'The New York Times/Siena College',
#     'Noble Predictive Insights': 'OH Predictive Insights',
#     'McLaughlin': 'McLaughlin & Associates'
# }

In [9]:
pollsters_short = np.unique(approval['pollster'])
pollsters_long = np.unique(approval['display_name'])

In [10]:
pollsters_short, pollsters_long

(array(['American Pulse', 'AtlasIntel', 'Beacon/Shaw', 'Big Data Poll',
        'Clarity', 'Cygnal', 'Echelon Insights', 'Emerson',
        'Fabrizio Ward', 'Fabrizio/Impact', 'Hart/POS', 'J.L. Partners',
        'McLaughlin', 'NewsNation/Decision Desk HQ',
        'Noble Predictive Insights', 'Normington, Petts & Associates',
        'Public Opinion Strategies', 'Quantus Insights', 'RMG Research',
        'RealClear Opinion Research', 'Siena/NYT', 'Verasight', 'Winston',
        'Yale Youth Poll', 'YouGov', 'co/efficient'], dtype=object),
 array(['American Pulse Research & Polling', 'AtlasIntel',
        'Beacon Research/Shaw & Co. Research', 'Big Data Poll',
        'Clarity Campaign Labs', 'Cygnal', 'Echelon Insights',
        'Emerson College', 'Fabrizio Ward',
        'Fabrizio, Lee & Associates/Impact Research',
        'Hart Research Associates/Public Opinion Strategies',
        'J.L. Partners', 'McLaughlin & Associates', 'NewsNation',
        'Noble Predictive Insights', 'Norm

In [11]:
morris_to_silver_dict = {short: long for short, long in zip(pollsters_short, pollsters_long)}
morris_to_silver_dict

{'American Pulse': 'American Pulse Research & Polling',
 'AtlasIntel': 'AtlasIntel',
 'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
 'Big Data Poll': 'Big Data Poll',
 'Clarity': 'Clarity Campaign Labs',
 'Cygnal': 'Cygnal',
 'Echelon Insights': 'Echelon Insights',
 'Emerson': 'Emerson College',
 'Fabrizio Ward': 'Fabrizio Ward',
 'Fabrizio/Impact': 'Fabrizio, Lee & Associates/Impact Research',
 'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
 'J.L. Partners': 'J.L. Partners',
 'McLaughlin': 'McLaughlin & Associates',
 'NewsNation/Decision Desk HQ': 'NewsNation',
 'Noble Predictive Insights': 'Noble Predictive Insights',
 'Normington, Petts & Associates': 'Normington, Petts & Associates',
 'Public Opinion Strategies': 'Public Opinion Strategies',
 'Quantus Insights': 'Quantus Insights',
 'RMG Research': 'RMG Research',
 'RealClear Opinion Research': 'RealClear Opinion Research',
 'Siena/NYT': 'The New York Times/Siena College',
 'Verasight': 'The Winston Group

In [12]:
morris_to_silver_dict.update({
    'HarrisX/Harris Poll':'Harris Insights & Analytics',
    'Clarity': 'Clarity Campaign Labs',
    'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
    'Monmouth': 'Monmouth University',
    'HarrisX': 'Harris Insights & Analytics',
    'UMass Amherst/YouGov': 'YouGov',
    'Emerson': 'Emerson College',
    'Marquette Law School': 'Marquette University Law School',
    'NewsNation/Decision Desk HQ': 'NewsNation',
    'CNN/SSRS': 'SSRS',
    'Suffolk': 'Suffolk University',
    'Harris Poll': 'Harris Insights & Analytics',
    'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
    'Winston': 'Verasight',
    'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
    'Quinnipiac': 'Quinnipiac University',
    'Siena/NYT': 'The New York Times/Siena College',
    'Noble Predictive Insights': 'OH Predictive Insights',
    'McLaughlin': 'McLaughlin & Associates',
})

In [13]:
approval['pollster'] = approval['pollster'].replace(morris_to_silver_dict)
approval = approval.rename({'population':'population_empty'}, axis='columns')
approval = approval.rename({'population_full':'population'}, axis='columns')
approval.head(7)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin & Associates,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178.0,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184.0,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
5,89530,294,McLaughlin & Associates,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,40.0,48.0
6,89555,1699,Fabrizio Lee & Associates/Impact Research,42.0,AARP,NaN,NaN,NaN,"Fabrizio, Lee & Associates/Impact Research",Online Opt-In Panel/Text-to-Web,...,NaN,https://www.aarp.org/content/dam/aarp/research...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,43.0,43.0


In [14]:
approval.shape

(103, 36)

In [15]:
approval = approval[approval['partisan'].isna()]# [pd.to_datetime(approval['end_date']) >= pd.to_datetime('2023-12-01')]
approval.shape

(82, 36)

In [16]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [17]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [18]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(80, 36)

In [19]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval.shape

(80, 38)

In [20]:
approval = approval[(~approval['sample_size'].isna())]
approval.shape

(79, 38)

In [21]:
approval['pollster'].value_counts()

pollster
YouGov                                                       17
Cygnal                                                        9
Echelon Insights                                              8
McLaughlin & Associates                                       7
RMG Research                                                  7
Clarity Campaign Labs                                         5
Fabrizio Lee & Associates/Impact Research                     3
Emerson College                                               3
The Winston Group                                             3
AtlasIntel                                                    2
Quantus Insights                                              2
Hart Research Associates/Public Opinion Strategies            2
Normington, Petts & Associates                                1
RealClear Opinion Research                                    1
OH Predictive Insights                                        1
Beacon Research/Shaw & Co. Rese

In [22]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [23]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [24]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [25]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [26]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['dem_adj'] = df['dem'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['rep_adj'] = df['rep'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['dem_adj'] = df['dem']
        df['rep_adj'] = df['rep']
    return df

In [27]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,Predictive Plus-Minus,dem_adj,rep_adj
9,89579,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.75,37.25
11,89579,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.75,41.25
23,89690,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.75,36.25
24,89690,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.75,42.25
39,89897,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.75,42.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,90035,1503,Fabrizio Ward,2182.0,Pinpoint Policy Institute,NaN,NaN,NaN,Fabrizio Ward,Text-to-Web,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,42.0,NaN,NaN,42.00,42.00
13,89658,379,Public Opinion Strategies,2255.0,Paragon Health Institute,NaN,NaN,NaN,Public Opinion Strategies,Live Phone,...,U.S. House,Generic Ballot,2026-11-03,general,44.0,46.0,Public Opinion Strategies,-0.246974,46.07,43.93
47,90057,1624,J.L. Partners,1997.0,DailyMail.com,NaN,NaN,NaN,J.L. Partners,Online Opt-In Panel,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,35.0,J.L. Partners,0.197911,42.00,35.00
60,90198,1810,American Pulse Research & Polling,NaN,NaN,NaN,NaN,NaN,American Pulse Research & Polling,Live Phone/Online Opt-In Panel/Text-to-Web,...,U.S. House,Generic Ballot,2026-11-03,general,46.6,42.4,American Pulse Research & Polling,0.800463,46.60,42.40


In [28]:
approval = approval_adj.copy()
approval.shape

(79, 40)

In [29]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id')]
    ndup = data[~data.duplicated(subset='poll_id')]
    #try:
    dup_a = dup[dup['population'].isin(['lv'])]
#     except ValueError:
#         print(dup['population'].isin(['a']))
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id')]
    ndup = df[~df.duplicated(subset='poll_id')]
    dup_lv = dup[dup['population'].isin(['rv'])]
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [30]:
approval = approval.rename({'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [31]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [32]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,pred_plus_minus,dem_adj,rep_adj
9,89579,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.75,37.25
11,89579,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.75,41.25
23,89690,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.75,36.25
24,89690,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.75,42.25
39,89897,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.75,42.25


In [33]:
approval.shape

(79, 40)

In [34]:
# net: negative --> dem advantage, positive --> rep advantage
approval['net'] = approval['rep'] - approval['dem']
approval.head(1)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,seat_name,election_date,stage,dem,rep,Pollster,pred_plus_minus,dem_adj,rep_adj,net
9,89579,568,YouGov,352.0,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.75,37.25,2.0


In [35]:
approval['url'] = approval['url_crosstab'].fillna(approval['url_topline']).fillna(approval['url_article'].fillna(''))

In [36]:
approval_polls = approval[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'dem_adj', 'rep_adj', 'net', 'pred_plus_minus', 'poll_id', 'url', 'sponsors']]
approval_polls

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.75,37.25,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
11,YouGov,2025-02-16,2025-02-18,1451.0,rv,42.0,41.0,41.75,41.25,-1.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.75,36.25,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
24,YouGov,2025-04-05,2025-04-08,1558.0,rv,43.0,42.0,42.75,42.25,-1.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
39,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.75,42.25,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,Fabrizio Ward,2025-05-15,2025-05-19,800.0,rv,42.0,42.0,42.00,42.00,0.0,NaN,90035,https://pinpointpolicyinstitute.org/wp-content...,Pinpoint Policy Institute
13,Public Opinion Strategies,2025-02-25,2025-03-02,1000.0,rv,44.0,46.0,46.07,43.93,2.0,-0.246974,89658,https://paragoninstitute.org/medicaid/paragon-...,Paragon Health Institute
47,J.L. Partners,2025-06-06,2025-06-06,1006.0,rv,42.0,35.0,42.00,35.00,-7.0,0.197911,90057,https://www.dailymail.co.uk/news/article-14796...,DailyMail.com
60,American Pulse Research & Polling,2025-06-23,2025-06-25,633.0,rv,46.6,42.4,46.60,42.40,-4.2,0.800463,90198,https://drive.google.com/file/d/1fQWKf1QhBKMmx...,NaN


In [37]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    # state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    # linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    linear_weights = delta.map(lambda x: (1 - x/60) if x < 60 else 0)
    exp_weights = 0.1**(delta/(45))
    state_pivot['time_weights'] =  0.1 * linear_weights + 0.9 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual):
            return 0.1
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'rv':
            return 0.8
        elif population == 'a':
            return 0.5
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [38]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
68,Echelon Insights,2025-07-10,2025-07-14,1084.0,lv,45.0,41.0,45.00,41.00,-4.0,-0.216672,90309,https://echelonin.wpenginepowered.com/wp-conte...,NaN,0.012315,0.323578,1.181365,1.0,0.056447
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.75,37.25,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0.014976,0.000150,0.964021,0.5,0.000013
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.75,36.25,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0.015607,0.001842,0.964021,0.5,0.000166
39,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.75,42.25,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist,0.014764,0.015013,0.964021,0.8,0.002050
42,YouGov,2025-05-23,2025-05-26,1486.0,rv,43.0,41.0,42.75,41.25,-2.0,0.047635,90016,https://today.yougov.com/topics/politics/track...,The Economist,0.014419,0.021479,0.964021,0.8,0.002864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,Fabrizio Ward,2025-05-15,2025-05-19,800.0,rv,42.0,42.0,42.00,42.00,0.0,NaN,90035,https://pinpointpolicyinstitute.org/wp-content...,Pinpoint Policy Institute,0.010580,0.015013,0.100000,0.8,0.000152
13,Public Opinion Strategies,2025-02-25,2025-03-02,1000.0,rv,44.0,46.0,46.07,43.93,2.0,-0.246974,89658,https://paragoninstitute.org/medicaid/paragon-...,Paragon Health Institute,0.011829,0.000277,1.209225,0.8,0.000038
47,J.L. Partners,2025-06-06,2025-06-06,1006.0,rv,42.0,35.0,42.00,35.00,-7.0,0.197911,90057,https://www.dailymail.co.uk/news/article-14796...,DailyMail.com,0.011864,0.037711,0.858783,0.8,0.003686
60,American Pulse Research & Polling,2025-06-23,2025-06-25,633.0,rv,46.6,42.4,46.60,42.40,-4.2,0.800463,90198,https://drive.google.com/file/d/1fQWKf1QhBKMmx...,NaN,0.009411,0.128031,0.200000,0.8,0.002312


In [39]:
approval_polls['end_date'].min()

'2024-12-11'

In [40]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 20), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['rep_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['dem_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['rep_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['dem_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Republicans':yes_averages, 'Democrats':no_averages}), pd.DataFrame({'Date':date_range, 'Republicans':yes_stdevs, 'Democrats':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [41]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Republicans,Democrats
0,2025-01-20,47.707765,44.677207
1,2025-01-21,47.707429,44.677135
2,2025-01-22,47.707087,44.677042
3,2025-01-23,47.706738,44.676927
4,2025-01-24,47.706384,44.676788
...,...,...,...
195,2025-08-03,42.445845,45.897389
196,2025-08-04,42.002430,45.613530
197,2025-08-05,42.001523,45.615243
198,2025-08-06,41.999333,45.616555


In [42]:
polls_df.shape

(75, 19)

In [43]:
display_df = polls_df[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'net',
                      'total_weights', 'url', 'sponsors']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 'total_weights':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population'].map(lambda x: x.upper()).astype(str)
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net
0,"<a href=""https://today.yougov.com/topics/polit...",2025-08-01,2025-08-04,1528 RV,1.172158e-01,43.7,38.4,-5.3
1,"<a href=""https://static1.squarespace.com/stati...",2025-07-24,2025-07-31,1018 RV,6.887760e-02,51.0,43.0,-8.0
2,"<a href=""https://today.yougov.com/topics/polit...",2025-07-25,2025-07-28,1610 RV,8.644744e-02,43.0,41.0,-2.0
3,"<a href=""https://docs.google.com/spreadsheets/...",2025-07-21,2025-07-22,1400 RV,9.780856e-02,44.0,41.5,-2.5
4,"<a href=""https://today.yougov.com/topics/polit...",2025-07-18,2025-07-21,1551 RV,6.113301e-02,44.0,41.0,-3.0
...,...,...,...,...,...,...,...,...
70,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.381566e-06,40.0,48.0,8.0
71,"<a href=""https://napolitannews.org/posts/gener...",2025-01-15,2025-01-16,1000 RV,2.541602e-06,44.0,51.0,7.0
72,"<a href=""https://www.cygn.al/wp-content/upload...",2025-01-09,2025-01-12,1500 LV,6.977820e-06,44.6,46.8,2.2
73,"<a href=""https://mclaughlinonline.com/pols/wp-...",2024-12-11,2024-12-16,1000 LV,1.610788e-07,42.0,47.0,5.0


In [44]:
display_df.to_csv('generic_ballot_polls_display_table.csv')

In [45]:
approval_avg['Net'] = approval_avg['Republicans'] - approval_avg['Democrats']
approval_avg

,Date,Republicans,Democrats,Net
0,2025-01-20,47.707765,44.677207,3.030558
1,2025-01-21,47.707429,44.677135,3.030294
2,2025-01-22,47.707087,44.677042,3.030044
3,2025-01-23,47.706738,44.676927,3.029811
4,2025-01-24,47.706384,44.676788,3.029596
...,...,...,...,...
195,2025-08-03,42.445845,45.897389,-3.451543
196,2025-08-04,42.002430,45.613530,-3.611100
197,2025-08-05,42.001523,45.615243,-3.613721
198,2025-08-06,41.999333,45.616555,-3.617221


In [46]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Republicans_x'] + 1.96*approval_trendline['Republicans_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Republicans_x'] - 1.96*approval_trendline['Republicans_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Democrats_x'] + 1.96*approval_trendline['Democrats_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Democrats_x'] - 1.96*approval_trendline['Democrats_y']
approval_trendline = approval_trendline.rename({
    'Republicans_x':'approve_mean',
    'Democrats_x':'disapprove_mean',
}, axis='columns').drop(['Republicans_y', 'Democrats_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-20,47.707765,44.677207,51.434214,43.981316,45.465268,43.889146
1,2025-01-21,47.707429,44.677135,51.434530,43.980327,45.465298,43.888972
2,2025-01-22,47.707087,44.677042,51.434988,43.979185,45.465336,43.888749
3,2025-01-23,47.706738,44.676927,51.435600,43.977876,45.465383,43.888472
4,2025-01-24,47.706384,44.676788,51.436380,43.976388,45.465440,43.888137
...,...,...,...,...,...,...,...
195,2025-08-03,42.445845,45.897389,48.095321,36.796370,52.237014,39.557763
196,2025-08-04,42.002430,45.613530,47.816400,36.188460,51.765577,39.461483
197,2025-08-05,42.001523,45.615243,47.806454,36.196591,51.765017,39.465470
198,2025-08-06,41.999333,45.616555,47.796304,36.202363,51.764082,39.469027


In [47]:
# remember: approve = Republicans, disapprove = Democrats - kept for DataWrapper setup simplicity
# Export generic ballot averages
approval_trendline.to_csv('generic_ballot_stats.csv')

In [48]:
# line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# # scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
# line.update_traces(hovertemplate=None)
# line.update_layout(hovermode='x unified')
# # scatter.update_traces(hovertemplate=None, hoverinfo='skip')
# fig_approve_CI = go.Figure([
#     go.Scatter(
#         name='Approve CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Approve CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])
# fig_disapprove_CI = go.Figure([
#     go.Scatter(
#         name='Disapprove CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Disapprove CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])

# fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
# fig.update_layout(
#     title='Congressional Approval Rating',
#     xaxis_title='Date',
#     yaxis_title='%',
#     legend_title='Legend',
#     hovermode='x unified'
# )
# fig.show()

In [50]:
approval_polls.head()

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.75,37.25,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
11,YouGov,2025-02-16,2025-02-18,1451.0,rv,42.0,41.0,41.75,41.25,-1.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.75,36.25,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
24,YouGov,2025-04-05,2025-04-08,1558.0,rv,43.0,42.0,42.75,42.25,-1.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist
39,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.75,42.25,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist


In [51]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['rep_adj'] - data['dem_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [53]:
net_approval = net_col(approval_polls, datetime.date(2025, 1, 20))
net_approval

,Date,net,net_std
0,2025-01-20,3.030558,2.276410
1,2025-01-21,3.030294,2.276860
2,2025-01-22,3.030044,2.277412
3,2025-01-23,3.029811,2.278076
4,2025-01-24,3.029596,2.278861
...,...,...,...
195,2025-08-03,-3.451543,3.600369
196,2025-08-04,-3.611100,3.410829
197,2025-08-05,-3.613721,3.410089
198,2025-08-06,-3.617221,3.409253


In [54]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-20,3.030558,2.276410,7.492322,-1.431206
1,2025-01-21,3.030294,2.276860,7.492939,-1.432351
2,2025-01-22,3.030044,2.277412,7.493772,-1.433684
3,2025-01-23,3.029811,2.278076,7.494840,-1.435218
4,2025-01-24,3.029596,2.278861,7.496164,-1.436972
...,...,...,...,...,...
195,2025-08-03,-3.451543,3.600369,3.605180,-10.508266
196,2025-08-04,-3.611100,3.410829,3.074125,-10.296325
197,2025-08-05,-3.613721,3.410089,3.070055,-10.297496
198,2025-08-06,-3.617221,3.409253,3.064914,-10.299357


In [55]:
net_approval.to_csv('net_generic_ballot.csv')